In [1]:
!pip install sentence_transformers
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

working_folder='/content/drive/My Drive/TransformersCode/03-restaurant/chatbot/'


QA_file_path=working_folder +'AR_QA.csv'

df = pd.read_csv(QA_file_path)

df.head()

,question,answer
0,ما هي المكونات اللازمة لتحضير بالو شاهي؟,دقيق مائدة، زبادي، زيت، سكر
1,ما هي المكونات المطلوبة لعمل بالو شاهي؟,دقيق مائدة، زبادي، زيت، سكر
2,مما يتكون بالو شاهي؟,دقيق مائدة، زبادي، زيت، سكر
3,ما هو نوع الحمية الغذائية ل بالو شاهي؟,نباتي
4,هل بالو شاهي نباتي أم غير نباتي؟,نباتي


In [4]:
import faiss

QAfaiss_file_path=working_folder + 'QA_faiss_index.bin'
index = faiss.read_index(QAfaiss_file_path)

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v1')

def get_embeddings(texts):
    embeddings = model.encode(texts, convert_to_tensor=True)
    return embeddings.numpy()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense%2Fconfig.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [6]:
def get_answer(query, df, index):

    query_embedding = get_embeddings([query])

    faiss.normalize_L2(query_embedding)

    _, indices = index.search(query_embedding, 1)

    nearest_question_idx = indices[0][0]
    return df.iloc[nearest_question_idx]['answer']

In [7]:
query = "ما هو النظام الغذائي ل دجاج تيكا؟"
answer = get_answer(query, df, index)
print("Answer:", answer)

Answer: غير نباتي


In [8]:
query = "ما مكونات دجاج تيكا؟"
answer = get_answer(query, df, index)
print("Answer:", answer)

Answer: دجاج، خبز القمح الكامل، دقيق الأرز، مسحوق جارام ماسالا، بيضة كاملة


In [9]:
query = "ما هي مكونات دجاج تيكا؟"
answer = get_answer(query, df, index)
print("Answer:", answer)

Answer: دجاج، خبز القمح الكامل، دقيق الأرز، مسحوق جارام ماسالا، بيضة كاملة


In [10]:
query = "مما يتكون دجاج تيكا؟"
answer = get_answer(query, df, index)
print("Answer:", answer)

Answer: دجاج، خبز القمح الكامل، دقيق الأرز، مسحوق جارام ماسالا، بيضة كاملة


In [11]:
query = "ما ذا يوجد مكونات في وجبة دجاج تيكا؟"
answer = get_answer(query, df, index)
print("Answer:", answer)

Answer: دجاج، خبز القمح الكامل، دقيق الأرز، مسحوق جارام ماسالا، بيضة كاملة
